In [ ]:
import numpy as np
import torch
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt

### Funcioncitas

In [ ]:
def broadcast_0darray(a:np.ndarray, times_x, times_y):
    a = np.repeat(a[:, np.newaxis], times_x, axis=0)
    a = np.repeat(a[:, np.newaxis], times_y, axis=1).squeeze()
    return a

In [ ]:
def calculate_weighted_matrix_multiplication(W:torch.Tensor, M_1: torch.Tensor, M_2: torch.Tensor):
    a = M_1.shape[1]
    WM_1 = W[:a,a:]@M_1.transpose(0,1)
    WM_1M_2 = WM_1.transpose(0,1)@M_2.transpose(0,1)
    return WM_1M_2

In [ ]:
def calculate_linear_weighted_features(M:torch.Tensor, w: torch.Tensor):
    """ Calcula Suma(i=1, a)wi Uui"""
    if M.shape[1] != w.shape[0]:
        raise Exception("calculate_linear_weighted_features: las matrices no se pueden multiplicar. Chequear la dimension.")
    else:
        return (M@w.reshape(-1,1))

In [ ]:
def calculate_weighted_interactions(M: torch.Tensor, W: torch.Tensor):
    """ Calcula las interacciones de cada fila de M ponderadas con W.
    Es una implementacion de la siguiente formula
    Suma(i=1, a-1)Suma(j=i+1, a) W_ij Uui Uuj
    """
    if M.shape[1]!=W.shape[0]:
        raise Exception("calculate_weighted_interactions: Las matrices M y W no se pueden multiplicar.")
    if W.shape[0]!=W.shape[1]:
        raise Exception("calculate_weighted_interactions: La matriz de pesos no es cuadrada.")
    
    M_interactions = calculate_interactions(M)
    upper_W = get_upper_triangle_matrix(W)
    M_weighted_interactions = (M_interactions@upper_W).reshape(-1,1)
    return M_weighted_interactions

In [ ]:
def get_upper_triangle_matrix(M: torch.Tensor):
    """Devuelve los elementos de la triangular superior sin la diagonal. 
    Esos elementos se aplastar a un array 1D
    Si M es la matriz de 3x3
    1 2 3
    4 5 6
    7 8 9
    La salida es el array [2,3,6]
    """
    if (M.shape[0]!=M.shape[1]):
       raise Exception("get_upper_triangle_matrix: La matriz no es cuadrada")
    else:
        out_size = M.shape[0]
        return M[np.triu_indices(out_size, k = 1)]

In [ ]:
def calculate_interactions(M: torch.Tensor):
    interaction = PolynomialFeatures(include_bias=False, interaction_only=True)
    M_interactions = interaction.fit_transform(M)
    M_interactions = M_interactions[:,M.shape[1]:] # Me quedo unicamente con las interacciones de tipo xy, xz, yz
    return M_interactions

In [ ]:
def init_indicator_matrix(ratings: torch.Tensor):
    """ Recibe la matriz de interacciones y
    devuelve la matriz indicadora del mismo tamaño que tiene
    0 donde la matriz de interacciones tiene -1 y 1 en todos los demás lugares
    """
    indicator = torch.ones((ratings.shape[0], ratings.shape[1]))
    indicator[np.where(ratings == -1)] = 0
    return indicator

In [ ]:
def plot_loss(loss_tracker: list):
    x = np.arange(0,len(loss_tracker)*100, 100)[1:]
    y = loss_tracker[1:]

    plt.plot(x, y, linewidth=2)
    plt.grid(alpha=.4)
    plt.xlabel("Epoch")
    plt.ylabel("MSE")
    plt.show()

In [ ]:
def entrenar_regresion(ratings, U, V, W, p, q, z, lr = 0.01, epochs = 10**6):
    loss_tracker = []
    L_anterior = 0
    L_nuevo = 0

    for i in range(epochs):
        L_anterior = L_nuevo
        # Forward Pass
        p_broad = torch.broadcast_to(p, (-1, n_items))
        q_broad = torch.broadcast_to(q, (-1, n_users)).transpose(0,1)
        z_broad = torch.broadcast_to(z, (n_users, n_items))
        R_uv = z_broad + p_broad + q_broad + calculate_weighted_matrix_multiplication(W,U,V)
        L_nuevo = ((ratings[ratings!=-1]-R_uv[ratings!=-1])**2).mean()/2 # Calculo función perdida sólo para los datos conocidos
        L_diferencia = (L_anterior - L_nuevo).item()

        mae = (torch.absolute(ratings[ratings!=-1]-R_uv[ratings!=-1])).mean()

        if (i%500==0):
            print("Epoch {} Loss: {}  MAE: {}".format(i, L_nuevo, mae), end = "\r")
        if (i%1000==0):
            loss_tracker.append(L_nuevo.item())

        if (L_diferencia < 0.0000001)&(i>100):
            print("\nEarly stopping en epoch {}".format(i))
            break

        delta_uv = indicator*(R_uv-ratings)
        # Update z
        z = z - lr*(delta_uv.mean())
        # Update p
        p = p - lr*(delta_uv.mean(axis=1).reshape(n_users,-1))
        # Update q
        q = q - lr*(delta_uv.mean(axis=0).reshape(n_items,-1))
        # Update U
        U = U - lr * (delta_uv@(W[:a,a:]@V.transpose(0,1)).transpose(0,1))/(n_items*b)
        # Update V
        V = V - lr * ((W[:a,a:]@U.transpose(0,1))@delta_uv).transpose(0,1)/(n_users*a)
        # Update W (O la parte de W que se va a usar)
        W[:a,a:] = W[:a,a:] - lr * ((delta_uv.transpose(0,1)@U).transpose(0,1)@V)/(n_users*n_items)
    
    acc = 1-torch.absolute(torch.round(R_uv[ratings!=-1])-ratings[ratings!=-1]).mean()
    print("Accuracy: {}".format(acc))
    plot_loss(loss_tracker)

### Proto Recomendador
Esta notebook está basada en el paper A Recommendation Model Based on Deep Neural Network del autor LIBO ZHANG

In [ ]:
n_users = 50
n_items = 100
# TODO: Si a no es igual a b se rompe. Ver por qué
a = 20
b = 20
l = a + b
sparce_rate = .9 # Qué tan dispersa es la matriz de ratings

# Inicializo la matriz de interacciones y pongo -1 en muchos lugares
# El -1 representa las interacciones que no conocemos o no sucedieron. Ej: las peliculas que no vio el usuario.
ratings = np.random.randint(0, 2, n_users*n_items).reshape(n_users,n_items)
aux = np.random.rand(ratings.shape[0], ratings.shape[1])
ratings[aux<sparce_rate] = -1

### Inicializo algunos pesos

In [ ]:
W = np.random.rand(l,l)
w = np.random.rand(l)

U = np.random.rand(n_users, a)
V = np.random.rand(n_items, b)

In [ ]:
p = calculate_linear_weighted_features(U,w[:a]) + calculate_weighted_interactions(U, W[:a,:a])
p.shape

In [ ]:
q = calculate_linear_weighted_features(V,w[a:]) + calculate_weighted_interactions(V,W[a:,a:])
q.shape

In [ ]:
z = np.random.rand(1)
z_broad = broadcast_0darray(z, times_x=n_users, times_y=n_items).shape

### Entrenamiento Regresión Cuadrática

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

In [ ]:
p = torch.from_numpy(p).to(device)
q = torch.from_numpy(q).to(device)
z = torch.from_numpy(z).to(device)
W = torch.from_numpy(W).to(device)
w = torch.from_numpy(w).to(device)
U = torch.from_numpy(U).to(device)
V = torch.from_numpy(V).to(device)
ratings = torch.from_numpy(ratings)
indicator = init_indicator_matrix(ratings)
ratings = ratings.to(device)
indicator = indicator.to(device)

In [ ]:
%time entrenar_regresion(ratings, U, V, W, p, q, z)